<br>
<font face="Helvetica" size="5"><b>VegMapper</b></font>
<br><br>
<font face="Helvetica" size="3">License Terms

Copyright (c) 2019, California Institute of Technology ("Caltech").  U.S. Government sponsorship acknowledged.

All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
* Redistributions must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
* Neither the name of Caltech nor its operating division, the Jet Propulsion Laboratory, nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

</font>
<br>

In [ ]:
#load in Python libraries
import ipywidgets as ipw
from ipyfilechooser import FileChooser
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [ ]:
#set up Python/R dual functionality for notebook
%load_ext rpy2.ipython

In [ ]:
%%R
#initialize R cell and load in necessary libraries
library(rgdal)
library(arm)
library(gdalUtils)
library(raster)

<br><font face="Helvetica" size="3">User inputs:</font>

In [ ]:
print("\n")
##### FILES #####
printmd('<font face="Helvetica" size="4"><b>FILE INPUTS:<b>')
#all GIS data in lat lon WGS84 unless specified
#path to comma-delimited file, must have cols 'latitude', 'longitude', 'class'
fc_in_points = FileChooser()
fc_in_points.use_dir_icons = True
fc_in_points.title = '<font face="Helvetica" size="2"><b>Please select the validation points (csv file):</b>'
display(fc_in_points)

#remote sensing stack in ENVI flat binary format, get stack info
fc_in_stack = FileChooser()
fc_in_stack.use_dir_icons = True
fc_in_stack.title = '<font face="Helvetica" size="2"><b>Please select the remote sensing stack (NOT the .hdr file):</b>'
display(fc_in_stack)

#path to output comma-delimited file, same as in_points with appended remote sensing values
printmd('<font face="Helvetica" size="2"><b>Name of output validation csv appended remote sensing values (please include .csv):<b>')
printmd('<font face="Helvetica" size="2">NOTE: there is no need to press enter to confirm the name')
name_out_points = ipw.Text(
    value='',
    placeholder='Type something',
    description='File name:',
    disabled=False
)
display(name_out_points)

#name of the output map in ENVI flat binary format
printmd('<font face="Helvetica" size="2"><b>Name of output map in ENVI flat binary format:<b>')
printmd('<font face="Helvetica" size="2">NOTE: there is no need to press enter to confirm the name')
name_out_pred = ipw.Text(
    value='',
    placeholder='Type something',
    description='File name:',
    disabled=False
)
display(name_out_pred)

#name of output GeoTIFF
printmd('<font face="Helvetica" size="2"><b>Name of output GeoTIFF (please include .tif):<b>')
printmd('<font face="Helvetica" size="2">NOTE: there is no need to press enter to confirm the name')
name_out_tif = ipw.Text(
    value='',
    placeholder='Type something',
    description='File name:',
    disabled=False
)
display(name_out_tif)


#--------------------------------------------------------------#


print("\n")
##### PARAMETERS #####
printmd('<font face="Helvetica" size="4"><b>PARAMETER INPUTS:<b>')
printmd('<font face="Helvetica" size="2">NOTE: when probability is larger than threshold, we say that oil palm is present')
#buffer
value_buffer = ipw.BoundedIntText(
    value=1,
    min=0,
    max=5,
    step=1,
    description='# of cells:',
    disabled=False
)

#threshold
value_threshold = ipw.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.05,
    description='Float:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

#prior
status_use_prior = ipw.Checkbox(
    value=False,
    description='Use prior in model?',
    disabled=False
)

accordion = ipw.Accordion(children=[value_buffer, value_threshold, status_use_prior])
accordion.set_title(0, 'Buffer size')
accordion.set_title(1, 'Threshold for logistic model')
accordion.set_title(2, 'Prior selection')
display(accordion)


<br><font face="Helvetica" size="3">Set inputs as variables:</font>

In [ ]:
in_points = fc_in_points.selected_filename
in_stack = fc_in_stack.selected_filename
out_points = name_out_points.value
out_pred = name_out_pred.value
out_tif = name_out_tif.value
buffer = value_buffer.value
threshold = value_threshold.value
use_prior = status_use_prior.value

print('User file inputs: ','\n', in_points, '\n', in_stack, '\n', out_points, '\n', out_pred, '\n', out_tif, '\n', buffer, '\n', threshold, '\n', use_prior)
#EDIT: could add exception handling here, other user input sections

<br><font face="Helvetica" size="3">Push file inputs to R:</font>

In [ ]:
%Rpush in_points in_stack out_points out_pred out_tif buffer threshold use_prior

In [ ]:
%%R
#test push success
push_file_variables = list(in_points, in_stack, out_points, out_pred, out_tif, buffer, threshold, use_prior)
print(push_file_variables)

<br><font face="Helvetica" size="3">TEMP: OTHER USER INPUTS W/O WIDGET:</font>

In [ ]:
%%R
##### PARAMETERS #####
#RUN: priors (variable order corresponds to order in which bands are read), currently Costa Rica
prior_mean = c(0.06491638, -26.63132179, 0.05590800, -29.64091620)
prior_scale = c(0.02038204, 7.58200324, 0.01686930, 8.73995422)
prior_mean_int = 1.99274801
prior_scale_int = 7.22600112

#lower and upper bounds for posterior credible intervals
lp = 0.025
up = 0.975

#--------------------------------------------------------------#

##### BAND INFO #####
bands = c(2,3,1,4)
stack_names = c("vcf", "c_rvi", "ndvi", "l_rvi_mosaic") #must be consistent with band names defined below

nodata = -9999 #NA value present in input bands: NEEDS TO BE A LIST OF NUMBERS IF NOT CONSISTENT ACROSS BANDS

#columns of the predictor variables to be used in this model (taken from pred csv)
    #column order indicated here is vcf, c_rvi, ndvi, l_rvi_mosaic, matching the priors above
index = c(13,14,15,16)
    #EDIT: find column indices using names of columns

#names for all the bands in your stack. 
bandNames = c('ndvi', 'vcf', 'c_rvi', 'lrvi', 'alpha', 'lambda', 'entropy', 'anisotropy', 'hh', 'mask', 'l_rvi_mosaic')

<hr><br>
<font face="Helvetica" size="5"><b>EXTRACT</b></font>
<br><br>
<font face="Helvetica" size="3">Objective: read remote sensing values at training points. The cell below creates a function to get intensity values from stack and executes extract routine:</font>

In [ ]:
%%R
getPixel= function(gdalObject, X, Y, buffer, ulX, ulY, cellSize, bands){
    nrow = dim(gdalObject)[1]
    ncol = dim(gdalObject)[2]
    rowOffset = ((ulY-Y)/cellSize) - buffer
    if(rowOffset<0 | (rowOffset+buffer+2) > nrow){
        return(NA)
    }
    colOffset = ((X-ulX)/cellSize) - buffer
    if(colOffset<0 | (colOffset+buffer+2) > ncol){
        return(NA)
    } 
    windowY = buffer+2
    windowX = windowY
    pixelValue = getRasterData(gdalObject, band=bands, offset=c(rowOffset, colOffset), region.dim=c(windowY, windowX))
    return(pixelValue)
}

#stack information
r_stack = stack(in_stack)
res = xres(r_stack)
r_extent = r_stack@extent
ulX = r_extent@xmin
ulY = r_extent@ymax
# s_info = GDALinfo(in_stack) also works, lacks ulY

#grab stack
gdalObj = new("GDALDataset", in_stack)

#append remote sensing information to point table and write to cvs file
inData <- read.csv(in_points, header=TRUE)
numPoints <- nrow(inData)

header <- c(colnames(inData), stack_names)
write.table(x=t(header), file=out_points, append=FALSE, col.names=FALSE, row.names=FALSE, sep=",")

print("Extracting values for...")
for(i in 1:numPoints){
    allBands <- rep(NA, length(bands))
    for (j in 1:length(bands)){
        oneBand = getPixel(gdalObj, inData$longitude[i], inData$latitude[i], buffer, ulX, ulY, res, bands[j])
        w = which (oneBand == nodata[j])
        oneBand[w]<-NA
        allBands[j] = mean(oneBand, na.rm=TRUE)
        if(i==1) print(stack_names[j])
    }
  mydata <- data.frame(t(as.vector(allBands)))
  colnames(mydata) <- stack_names
  newRow = cbind(inData[i,],mydata)
  write.table(x=newRow, file=out_points, append=TRUE, col.names=FALSE, row.names=FALSE, sep=",") 
}

<hr><br>
<font face="Helvetica" size="5"><b>RUN</b></font>
<br><br>
<font face="Helvetica" size="3">Objective: fit Bayesian model, calculate posteriors and confusion matrix. The cell below creates the model and executes the prediction, constructs a confusion matrix, calculates the prediction accuracy/posterior CI, calculates/builds posteriors for subsequent runs, and prints the result:</font>

In [ ]:
%%R
#impute missing values by variable means
data = read.csv(out_points)
for (i in which(sapply(data, is.numeric))) {
    for (j in which(is.na(data[, i]))) {
        data[j, i] <- mean(data[data[, "my_class"] == data[j, "my_class"], i],  na.rm = TRUE)
    }
}

#true_label: 1 for oil_palm and 0 for non oil_palm
true_label = 1*(data$my_class == 'oil_palm') #EDIT: why this --> http://127.0.0.1:54098/notebooks/sample-run/example_analysis.ipynb

#transform interested variables into a matrix which would be used
x = as.matrix(data[, index])
all_names = names(data)
stack_names = all_names[index]
colnames(x) = stack_names

#build model by incorporating those variables
formula = as.formula(paste("true_label ~ ", paste(stack_names, collapse="+"),sep = ""))
use_data = as.data.frame(cbind(x, true_label))

#to specify prior
#if noninformative prior, use prior.mean=apply(x, 2, mean), prior.scale=Inf, prior.df=Inf
#if having a prior, set prior.mean=c(....), prior.scale=c(.....)
#length of prior mean and prior scale should be equal to the number of predictors
if(! use_prior){
    model = bayesglm(formula, data=use_data, family=binomial(link='logit'), prior.mean=apply(x, 2, mean), prior.scale=Inf, scale=FALSE)
}
if(use_prior){
    model = bayesglm(formula, data=use_data, family=binomial(link='logit'), 
                    prior.mean=prior_mean,
                    prior.scale=prior_scale,
                    prior.mean.for.intercept=prior_mean_int,
                    prior.scale.for.intercept=prior_scale_int,
            scale = FALSE)
}

#oil_palm prediction
class_prediction = 1*(model$fitted.values >= threshold) #if the fitted value is above the threshold, value is changed to binary 1
print(class_prediction)

#used instead of na.remove to get rid of NA values in 2018 validation dataset
true_label = true_label[!is.na(true_label)]

#generate confusion matrix
bayesian_conf_matrix = matrix(0,2,2)
bayesian_conf_matrix[1,1] = sum(class_prediction + true_label == 0)
bayesian_conf_matrix[2,2] = sum(class_prediction + true_label == 2)
bayesian_conf_matrix[1,2] = sum((class_prediction == 0) & (true_label == 1))
bayesian_conf_matrix[2,1] = sum((class_prediction == 1) & (true_label == 0))
rownames(bayesian_conf_matrix) = c("Predicted non-oil-palm", "Predicted oil-palm")
colnames(bayesian_conf_matrix) = c("Actual non-oil-palm", "Actual oil-palm")
print(bayesian_conf_matrix)

#overall accuracy of model
accu_bayes = sum(class_prediction == true_label) / nrow(data)
print("Overall accuracy:")
print(accu_bayes)

#EDIT: push values to Python and use numpy/matplotlib to display matrix

# approach posterior distributions of coefficients
# specify number of draws 
num_draw = 2000
post_dist = sim(model, n.sims=num_draw)
coef_matrix = coef(post_dist)

# calculate posterior credible intervals for coefficients
posterior_ci_coef = matrix(NA, ncol(x)+1, 2)
for (i in 1:(ncol(x)+1)){
    posterior_ci_coef[i, ] = unname(quantile(coef_matrix[, i], probs=c(lp, up), na.rm=TRUE))
}

# calculate posterior credible intervals for every data point
posterior_ci_data = matrix(NA, nrow(x), 2)
for(i in 1:nrow(x)){
    temp = as.numeric()
    for(j in 1:num_draw){
        temp[j] = 1 / (1 + exp(-coef_matrix[j, 1] - sum(coef_matrix[j, 2:(length(index)+1)] * x[i, ])))
    }
    posterior_ci_data[i, ] = unname(quantile(temp, probs=c(lp, up), na.rm=TRUE))
}

# build posterior objects for next run
posterior_mean = model$coefficients
posterior_scale = apply(coef(post_dist), 2, sd)

#print the posteriors, store them for later
#EDIT: not sure if this works with combinations of other stack variables than current build (need to test in future)
posterior_mean
intercept = posterior_mean[["(Intercept)"]]
numVars = length(posterior_mean)
posteriors = posterior_mean[2:numVars] #EDIT: does this need to be transformed into c() variable?

<hr><br>
<font face="Helvetica" size="5"><b>APPLY</b></font>
<br><br>
<font face="Helvetica" size="3">Objective: apply model fits to calculate OP3 for the area covered by the data stack. OP3 = oil palm probability presence, ranging between 0-1. The cell below applies the model to the stack and executes the predictive analysis, outputting the prediction surface in GeoTIFF and ENVI binary formats:</font>

In [ ]:
%%R
#generate dummy for Docker
    #only use env variables if creation fails (will normally return NULL but still create object)
# Sys.setenv(PROJ_LIB="/usr/bin/proj/")
# Sys.getenv("PROJ_LIB")
#dummy corresponds to one band from the original stack, used to save your output prediction map 
in_dummy = "temp_dummy"
gdal_translate(src_dataset=in_stack, dst_dataset=in_dummy, of="ENVI", b=1)

#determine bands to use
modelBands = rep(NA, length(stack_names))
for(i in 1:length(modelBands)){
    w = which(bandNames == stack_names[i])
    modelBands[i] = w
}

#create GIS objects
gdalObjStack = new("GDALDataset", in_stack)
gdalObjDummy = new("GDALDataset", in_dummy)
rasterWidth = ncol(gdalObjStack)
rasterRows = nrow(gdalObjStack)

#calculate prediction for each pixel and save
print("Checking a few values...")
for(i in 1:rasterRows){
    oneRasterLine = getRasterData(gdalObjStack, offset=c(i-1,0), region.dim=c(1, rasterWidth))
    hhBand = which(bandNames == "alos2_hh")
    pred = rep(-9999, rasterWidth)
    for(j in 1:rasterWidth){
        #hh = (20*log10(oneRasterLine[j, 1, hhBand])) -83
        hh = oneRasterLine[j, 1, hhBand] #gets hh value at each of the pixels
        #open water mask
        #if(is.na(hh) | hh < -20){ 
        #pred[j] = 0
        #} 
        #else{
            #select bands
            selectBands = oneRasterLine[j, 1, modelBands]
            z = (intercept + sum(posteriors * selectBands))
            pred[j] = exp(z)/(1+ exp(z))
            #z = (intercept + sum(posteriors * scaledBands))
            #pred[j] = exp(z)/(1+ exp(z))
            if ((i/100 == i%/%100) & j == 1000) print(z) #reality check on the model fits
        #}
    }
    #write one row to file
    putRasterData(gdalObjDummy, pred, offset=c(i-1, 0)) #place predicted line in raster into dummy
}
saveDataset(gdalObjDummy, out_pred)

#convert to GeoTiff
gdal_translate(src_dataset=out_pred, dst_dataset=out_tif, of="GTiff")